In [ ]:
from pathlib import Path

import jax
import jax.numpy as jnp
import torch
from matplotlib import pyplot as plt
from PIL import Image
from torch.utils import data

plt.rcParams["image.cmap"] = "gray"

# Training a Digit Classifier

In [ ]:
mnist_path = Path("~/.fastai/data/mnist_sample/").expanduser()

In [ ]:
if not mnist_path.exists():
    import fastbook

    fastbook.untar_data(fastbook.URLs.MNIST_SAMPLE)

In [ ]:
threes = list(mnist_path.glob("train/3/*.png"))
sevens = list(mnist_path.glob("train/7/*.png"))

In [ ]:
len(threes), len(sevens)

In [ ]:
im3_path = threes[1]
im3 = Image.open(im3_path)
im3

In [ ]:
jnp.array(im3)[4:9, 4:9]

Generally, when images are floats, the pixel values are expected to be between 0 and 1,
so we will also divide by 255 here

In [ ]:
stacked_threes = jnp.stack([jnp.array(Image.open(p)) for p in threes]) / 255
stacked_sevens = jnp.stack([jnp.array(Image.open(p)) for p in sevens]) / 255
stacked_threes.shape, stacked_sevens.shape

In [ ]:
plt.imshow(stacked_threes[0])

In [ ]:
valid_3s = (
    jnp.stack([jnp.array(Image.open(p)) for p in mnist_path.glob("valid/3/*.png")])
    / 255
)
valid_7s = (
    jnp.stack([jnp.array(Image.open(p)) for p in mnist_path.glob("valid/7/*.png")])
    / 255
)

## Comparing with the perfect 3 and 7

In [ ]:
mean_3 = jnp.mean(stacked_threes, axis=0)
plt.imshow(mean_3)

In [ ]:
mean_3[4:9, 4:9]

In [ ]:
mean_7 = jnp.mean(stacked_sevens, axis=0)
plt.imshow(mean_7)

In [ ]:
def mnist_distance(a: jnp.ndarray, b: jnp.ndarray):
    return jnp.mean(jnp.abs(a - b), axis=(-2, -1))

In [ ]:
example_3 = stacked_threes[0]

In [ ]:
mnist_distance(example_3, mean_3), mnist_distance(example_3, mean_7)

In [ ]:
valid_3_to_3 = mnist_distance(valid_3s, mean_3)
valid_3_to_7 = mnist_distance(valid_3s, mean_7)
valid_7_to_3 = mnist_distance(valid_7s, mean_3)
valid_7_to_7 = mnist_distance(valid_7s, mean_7)

How many 3s are detected as 3?

In [ ]:
percision_3 = jnp.sum(valid_3_to_3 < valid_3_to_7) / valid_3s.shape[0]
percision_3

In [ ]:
percision_7 = jnp.sum(valid_7_to_7 < valid_7_to_3) / valid_7s.shape[0]
percision_7

In [ ]:
(percision_3 + percision_7) / 2

When a image is claimed as 3, how much chance will it be accually 3?

In [ ]:
recall_3 = (
    percision_3
    * valid_3s.shape[0]
    / (percision_3 * valid_3s.shape[0] + (1 - percision_7) * valid_7s.shape[0])
)
recall_3

## SGD

### Making a simple network

In [ ]:
train_x = jnp.reshape(jnp.concatenate([stacked_threes, stacked_sevens]), (-1, 28 * 28))
train_y = jnp.expand_dims(
    jnp.concatenate(
        [jnp.ones(stacked_threes.shape[0]), jnp.zeros(stacked_sevens.shape[0])]
    ),
    axis=1,
)
train_x.shape, train_y.shape

JAX's random number generation is somewhat different from what Numpy or PyTorch has.
It focuses on the deterministic generation of sequences of pseudorandom numbers.

> Unlike the stateful pseudorandom number generators (PRNGs) that users of NumPy and SciPy may be accustomed to, JAX random functions all require an explicit PRNG state to be passed as a first argument.

https://jax.readthedocs.io/en/latest/jax.random.html

In [ ]:
key = jax.random.PRNGKey(996)

In [ ]:
def init_params(key, shape, std=1.0):
    return jax.random.normal(key, shape) * std

In [ ]:
def init_all_params(key):
    key_w, key_b = jax.random.split(key)
    weights = init_params(key_w, (28 * 28, 1))
    bias = init_params(key_b, (1,))
    return weights, bias

In [ ]:
def linear1(xb, params):
    w, b = params
    return xb @ w + b

In [ ]:
key, subkey = jax.random.split(key)
params = init_all_params(key)

In [ ]:
pred = linear1(train_x, params)
pred

In [ ]:
corrects = (pred > 0) == train_y
jnp.mean(corrects)

### Choosing the loss function

> The problem is that a small change in weights from `x_old` to `x_new` isn’t likely to cause any prediction to change, so `(y_new – y_old)` will almost always be 0. In other words, the gradient is 0 almost everywhere.

Instead of modifying the 0th weight, let's find the weight which is the most important.
That's because the 0th pixel is usually 0, and changing the 0th weight will not lead to any difference. If changing the most important weight is not making any difference, the loss function is indeed bad.

In [ ]:
pixels_mean = jnp.mean(train_x, axis=0)
significant_index = jnp.argmax(jnp.mean(train_x, axis=0)).item()
pixels_mean[0].item(), significant_index, pixels_mean[significant_index].item()

In [ ]:
params[0][significant_index]

In [ ]:
new_weights = params[0].at[significant_index].multiply(1.0001)

In [ ]:
pred_with_new_weights = linear1(train_x, (new_weights, params[1]))
jnp.mean((pred_with_new_weights > 0) == train_y)

> A very small change in the value of a weight will often not change the accuracy at all. This means it is not useful to use accuracy as a loss function.

We choose the (vector) distance between the true value and the prediction.

```python
trgts = tensor([1,0,1])
prds = tensor([0.9, 0.4, 0.2])
```

In [ ]:
def mnist_loss(predictions, targets):
    # The predictions should lie in [0, 1] range
    # normalized_pred = jax.nn.sigmoid(predictions)
    normalized_pred = predictions
    return jnp.mean((normalized_pred - targets) ** 2)

As you can see, the new loss function is better.

In [ ]:
mnist_loss(pred, train_y)

In [ ]:
mnist_loss(pred_with_new_weights, train_y)

> (The loss fucntion) is rather a compromise between our real goal and a function that can be optimized using its gradient.

### Optimization

#### Mini-batch

Torch tenser can be converted to `jnp.ndarray`.

In [ ]:
example_tensor = torch.Tensor([1.0, 2.0, 3.0])
example_jndarray = jnp.array(torch.Tensor([1.0, 2.0, 3.0]))
example_jndarray

But the reverse process is a little tricky. You should transfer the array with DLPack.

In [ ]:
import jax.dlpack

torch.from_dlpack(jax.dlpack.to_dlpack(example_jndarray))

# This will raise an error
# torch.tensor(example_jndarray)

Torch's `DataLoader` can be used with JAX: https://jax.readthedocs.io/en/latest/notebooks/Neural_Network_and_Data_Loading.html

In [ ]:
def numpy_collate(batch):
    if isinstance(batch[0], jnp.ndarray):
        return jnp.stack(batch)
    elif isinstance(batch[0], (tuple, list)):
        transposed = zip(*batch)
        return [numpy_collate(samples) for samples in transposed]
    else:
        return jnp.array(batch)

In [ ]:
class NumpyLoader(data.DataLoader):
    def __init__(self, dataset, *args, **kwargs):
        super().__init__(dataset, *args, collate_fn=numpy_collate, **kwargs)

In [ ]:
coll = range(15)
list(NumpyLoader(coll, batch_size=5))

In [ ]:
list(data.DataLoader(coll, batch_size=5))

In [ ]:
dset = zip(train_x, train_y)
valid_x = jnp.reshape(jnp.concatenate([valid_3s, valid_7s]), (-1, 28 * 28))
valid_y = jnp.expand_dims(
    jnp.concatenate([jnp.ones(valid_3s.shape[0]), jnp.zeros(valid_7s.shape[0])]),
    axis=1,
)
train_x.shape, train_y.shape
valid_dset = zip(valid_x, valid_y)

In [ ]:
dl = NumpyLoader(dset, batch_size=256)
valid_dl = NumpyLoader(valid_dset, batch_size=256)

In [ ]:
key, subkey = jax.random.split(key)
weights = init_params(subkey, (28 * 28, 1))
key, subkey = jax.random.split(key)
bias = init_params(subkey, (1,))

In [ ]:
# def train_epoch